In [ ]:
!pip install -q streamlit
!nvidia-smi
!pip install diffusers==0.17.0
!pip install transformers scipy ftfy accelerate
!pip install requests
!pip install pillow


Fri Jun 23 21:21:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    27W /  70W |   8373MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [239]:
%%writefile UI/pages/1_📈_Model_A.py
import streamlit as st
from transformers import AutoProcessor, AutoModelForCausalLM, BlipProcessor, BlipForConditionalGeneration
import requests
from PIL import Image
import numpy as np
import torch

from diffusers import StableDiffusionControlNetPipeline
from diffusers.utils import load_image

import cv2
from PIL import Image
import numpy as np
import os

from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch
from diffusers import UniPCMultistepScheduler
st.set_page_config(
    page_title="Realistic Vision V1.4 CUDA Blip Base",
    page_icon="📈",
)

st.title('Realistic Vision V1.4 CUDA Blip Base')
temp_prompt = ''

def imageurl_process(url):
    # submit image url into blip base processor
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")
    raw_image = Image.open(requests.get(url, stream=True).raw).convert('RGB')
    # unconditioned prompt generation
    text = "a photography of"
    inputs = processor(raw_image, text, return_tensors="pt").to("cuda")
    # conditioned prompt generation
    out = model.generate(**inputs)
    inputs = processor(raw_image, return_tensors="pt").to("cuda")
    out = model.generate(**inputs)
    temp_prompt = processor.decode(out[0], skip_special_tokens=True)
    # generate prompt and output
    st.info(temp_prompt)
    return temp_prompt

def prompt_process(text, url):
  model = "SG161222/Realistic_Vision_V1.4"
  controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
  pipe = StableDiffusionControlNetPipeline.from_pretrained(
      model, controlnet=controlnet, torch_dtype=torch.float16
  )

  pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

  pipe.enable_model_cpu_offload()

  low_threshold = 100
  high_threshold = 200

  starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
  repetitions = 20
  timings =np.zeros((repetitions,1))
  image = load_image(url)
  image = np.array(image)
  image = cv2.Canny(image, low_threshold, high_threshold)
  image = image[:, :, None]
  image = np.concatenate([image, image, image], axis=2)
  canny_image = Image.fromarray(image)
  add = ", best quality, extremely detailed"
  finalprompt = [text + add][:77]
  generator = [torch.Generator(device="cuda").manual_seed(2)]
  output = pipe(
    finalprompt,
    canny_image,
    negative_prompt=["monochrome, lowres, bad anatomy, worst quality, low quality"],
    generator=generator,
    num_inference_steps=20,
  )
  image = output.images[0]
  return image


if 'stage' not in st.session_state:
    st.session_state.stage = 0

def set_stage(stage):
    st.session_state.stage = stage

first_form = st.form('image_url')
url = first_form.text_input('Enter Image URL: ', '')
first_form.form_submit_button('Generate Prompt', on_click=set_stage, args=(1,))

if st.session_state.stage > 0:
    temp_prompt = imageurl_process(url)
    second_form = st.form('prompt')
    topic_text = second_form.text_input('Enter Prompt: ')
    second_form.form_submit_button('Generate Image', on_click=set_stage, args=(2,))
    if st.session_state.stage > 1:
        st.image(prompt_process(topic_text, url))
st.button('Reset', on_click=set_stage, args=(0,))






Overwriting UI/pages/1_📈_Model_A.py


In [238]:
%%writefile UI/pages/2_🌍_Model_B.py

import streamlit as st
from transformers import AutoProcessor, AutoModelForCausalLM, BlipProcessor, BlipForConditionalGeneration
import requests
from PIL import Image
import numpy as np
import torch

from diffusers import StableDiffusionControlNetPipeline
from diffusers.utils import load_image

import cv2
from PIL import Image
import numpy as np
import os

from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch
from diffusers import UniPCMultistepScheduler

st.set_page_config(
    page_title="Realistic Vision V1.4 CUDA Git Large Coco",
    page_icon="🌍",
)

st.title('Realistic Vision V1.4 CUDA Git Large Coco')
temp_prompt = ''

def imageurl_process(url):
    processor = AutoProcessor.from_pretrained("microsoft/git-base-coco")
    model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-coco")
    image = Image.open(requests.get(url, stream=True).raw)
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
    temp_prompt = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    st.info(temp_prompt)
    return temp_prompt

def prompt_process(text, url):
  model = "SG161222/Realistic_Vision_V1.4"
  controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
  pipe = StableDiffusionControlNetPipeline.from_pretrained(
      model, controlnet=controlnet, torch_dtype=torch.float16
  )

  pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

  pipe.enable_model_cpu_offload()

  low_threshold = 100
  high_threshold = 200

  starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
  repetitions = 20
  timings =np.zeros((repetitions,1))
  image = load_image(url)
  image = np.array(image)
  image = cv2.Canny(image, low_threshold, high_threshold)
  image = image[:, :, None]
  image = np.concatenate([image, image, image], axis=2)
  canny_image = Image.fromarray(image)
  add = ", best quality, extremely detailed"
  finalprompt = [text + add][:77]
  generator = [torch.Generator(device="cuda").manual_seed(2)]
  output = pipe(
    finalprompt,
    canny_image,
    negative_prompt=["monochrome, lowres, bad anatomy, worst quality, low quality"],
    generator=generator,
    num_inference_steps=20,
  )
  image = output.images[0]
  return st.image(image)

if 'stage' not in st.session_state:
    st.session_state.stage = 0

def set_stage(stage):
    st.session_state.stage = stage

first_form = st.form('image_url')
url = first_form.text_input('Enter Image URL: ', '')
first_form.form_submit_button('Generate Prompt', on_click=set_stage, args=(1,))

if st.session_state.stage > 0:
    temp_prompt = imageurl_process(url)
    second_form = st.form('prompt')
    topic_text = second_form.text_input('Enter Prompt: ')
    second_form.form_submit_button('Generate Image', on_click=set_stage, args=(2,))
    if st.session_state.stage > 1:
        st.image(prompt_process(topic_text, url))
st.button('Reset', on_click=set_stage, args=(0,))


Overwriting UI/pages/2_🌍_Model_B.py


In [237]:
%%writefile UI/pages/3_📊_Model_C.py

import streamlit as st
from transformers import AutoProcessor, AutoModelForCausalLM, BlipProcessor, BlipForConditionalGeneration
import requests
from PIL import Image
import numpy as np
import torch

from diffusers import StableDiffusionControlNetPipeline
from diffusers.utils import load_image

import cv2
from PIL import Image
import numpy as np
import os

from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch
from diffusers import UniPCMultistepScheduler

st.set_page_config(
    page_title="Photoreal 2.0 CUDA Git Large Coco",
    page_icon="📊",
)

st.title('Photoreal 2.0 CUDA Git Large Coco')
temp_prompt = ''

def imageurl_process(url):
    processor = AutoProcessor.from_pretrained("microsoft/git-base-coco")
    model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-coco")
    image = Image.open(requests.get(url, stream=True).raw)
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
    temp_prompt = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    st.info(temp_prompt)
    return temp_prompt

def prompt_process(text, url):
  model = "dreamlike-art/dreamlike-photoreal-2.0"
  controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
  pipe = StableDiffusionControlNetPipeline.from_pretrained(
      model, controlnet=controlnet, torch_dtype=torch.float16
  )

  pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

  pipe.enable_model_cpu_offload()

  low_threshold = 100
  high_threshold = 200

  starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
  repetitions = 20
  timings =np.zeros((repetitions,1))
  image = load_image(url)
  image = np.array(image)
  image = cv2.Canny(image, low_threshold, high_threshold)
  image = image[:, :, None]
  image = np.concatenate([image, image, image], axis=2)
  canny_image = Image.fromarray(image)
  add = ", best quality, extremely detailed"
  finalprompt = [text + add][:77]
  generator = [torch.Generator(device="cuda").manual_seed(2)]
  output = pipe(
    finalprompt,
    canny_image,
    negative_prompt=["monochrome, lowres, bad anatomy, worst quality, low quality"],
    generator=generator,
    num_inference_steps=20,
  )
  image = output.images[0]
  return st.image(image)

if 'stage' not in st.session_state:
    st.session_state.stage = 0

def set_stage(stage):
    st.session_state.stage = stage

first_form = st.form('image_url')
url = first_form.text_input('Enter Image URL: ', '')
first_form.form_submit_button('Generate Prompt', on_click=set_stage, args=(1,))

if st.session_state.stage > 0:
    temp_prompt = imageurl_process(url)
    second_form = st.form('prompt')
    topic_text = second_form.text_input('Enter Prompt: ')
    second_form.form_submit_button('Generate Image', on_click=set_stage, args=(2,))
    if st.session_state.stage > 1:
        st.image(prompt_process(topic_text, url))
st.button('Reset', on_click=set_stage, args=(0,))


Overwriting UI/pages/3_📊_Model_C.py


In [247]:
%%writefile UI/Home.py

import streamlit as st

st.write("# Welcome to a Live Demo of the Experimental Generative AI Models for Tile Generation! 👋")

st.sidebar.success("Select a model to run above.")

st.markdown(
    """
    **👈 Select a model** to experiment with recreating some images!
    ### Steps
    - Input an image url into the first form to generate a prompt
    - Customize the prompt or directly copy and paste it into the second form for image regeneration
    - For more information about the different models, check out this [presentation](https://drive.google.com/file/d/1cGuGW_FV0nmErdlhsaOx9vsiANP-3bf0/view?usp=drive_link)
    ### Overview
    - Model A is Realistic Vision V1.4 CUDA Blip Base (the best overall model)
    - Model B is Realistic Vision V1.4 CUDA Git Large Coco (the best disregarding NSFW scores)
    - Model C is Photoreal 2.0 CUDA Git Large Coco (the best disregarding any quality assesments)
"""
)

st.markdown(":green[Created By Nicole Chen]")


Overwriting UI/Home.py


In [248]:
!npm install localtunnel
!streamlit run UI/Home.py &>/content/logs.txt &
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.303s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities

34.168.25.84
npx: installed 22 in 1.232s
your url is: https://cute-mugs-warn.loca.lt
